In [5]:
import sys
sys.path.append('../src')

from visualization import *

# Load data
results_path = load_latest_results(station_filter="HAMILTON")
data = load_experiment_data(results_path)

# Create plots
fig, ax = plot_cdf_comparison(data["03_cdf_results"])
fig, ax = plot_event_scatter(data["03_rainfall_events_data"])

FileNotFoundError: [Errno 2] No such file or directory: 'data/results'